In [3]:
import pandas as pd
from pandas.io.formats.style import Styler
from pybatfish.client.session import Session
from pybatfish.datamodel import *
from pybatfish.datamodel.answer import *
from pybatfish.datamodel.flow import *
from pybatfish.util import get_html

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option("display.html.use_mathjax", False)

# UUID for CSS styles used by pandas styler.
# Keeps our notebook HTML deterministic when displaying dataframes
_STYLE_UUID = "pybfstyle"

bf = Session(host="localhost")

In [52]:
class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)


def show(df):
    """
    Displays a dataframe as HTML table.

    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """
    if isinstance(df, TableAnswer):
        df = df.frame()

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    display(
        MyStyler(df)
        .set_uuid(_STYLE_UUID)
        .format(get_html)
        .set_properties(**{"text-align": "left", "vertical-align": "top"})
    )

In [63]:
NETWORK_NAME = 'example'
SNAPSHOT_NAME = 'test_snapshot'

SNAPSHOT_PATH = 'test_snapshot'

bf.set_network(NETWORK_NAME)
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'test_snapshot'

In [65]:
bf.q.initIssues().answer().frame()

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context


In [58]:
inRoute1 = BgpRoute(network="100.0.0.0/24",
                    originatorIp="100.0.0.3",
                    originType="incomplete",
                    protocol="bgp")

result = bf.q.testRoutePolicies(
    policies = "POLICY",
    direction = "in",
    inputRoutes = [inRoute1]).answer().frame()

show(result)

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference,Trace
0,routera,POLICY,Network: 100.0.0.0/24AS Path: []Communities: []Local Preference: 0Metric: 0Next Hop IP: NoneOriginator IP: 100.0.0.3Origin Type: incompleteProtocol: bgpSource Protocol: NoneTag: 0Weight: 0,PERMIT,"Network: 100.0.0.0/24AS Path: [{'asns': [300], 'confederation': False}, {'asns': [400], 'confederation': False}]Communities: []Local Preference: 200Metric: 50Next Hop IP: NoneOriginator IP: 100.0.0.3Origin Type: incompleteProtocol: bgpSource Protocol: NoneTag: 0Weight: 0","AS Path: [] --> [300, 400]Local Preference: 0 --> 200Metric: 0 --> 50",Matched route-map POLICY clause 10


In [66]:
result = bf.q.bgpRib(nodes ='RouterC').answer().frame()
show(result)

,Node,VRF,Network,Status,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,AS_Path,Metric,Local_Pref,Communities,Origin_Protocol,Origin_Type,Originator_Id,Received_From_IP,Received_Path_Id,Cluster_List,Tunnel_Encapsulation_Attribute,Weight,Tag
0,routerc,default,100.0.0.0/24,BEST,ip 1.0.0.1,1.0.0.1,dynamic,bgp,10 300 400,50,200,,bgp,igp,10.0.0.1,1.0.0.1,None,None,None,0,None
